# Классы и датаклассы
Редко, когда в более-менее большом проекте, хватает типов данных, предоставляемых интерпретатором. Один из распространенных примеров - ORM (Object-Relation Mapping) - объектно-реляционное отображение. Это когда мы хотим связать поля в базе данных с объектом Python (не обязательно Python, конечно, просто мы с вами про Python говорим). Но даже и без БД мы можем хранить данные в экземплярах классов. Пусть, у нас будет некоторый класс `User`, в котором будут следующие атрибуты:

- `user_id`
- `name`
- `age`
- `email`

Стандартно наш класс User в Python можно оформить так:

In [1]:
class User:
    def __init__(self, user_id, name, age, email):
        self.user_id = user_id
        self.name = name
        self.age = age
        self.email = email

И затем, нам никто не мешает передавать экземпляр такого класса в какую-нибудь функцию:

In [2]:
def get_user_info(user: User) -> str:
    return f'Возраст пользователя {user.name} - {user.age}, ' \
           f'а email - {user.email}'


user_1: User = User(42, 'Vasiliy', 23, 'vasya_pupkin@pochta.ru')
print(get_user_info(user_1))

Возраст пользователя Vasiliy - 23, а email - vasya_pupkin@pochta.ru


Но, скажите, вас никогда не напрягали все эти `__init__(self)` и `self.something = something`? Меня напрягали всегда :) И, к счастью, видимо, не меня одного, потому что умные люди придумали датаклассы (`dataclasses`), которые убирают за "ширму" все эти `__init__(self)`. Смотрите, как лаконично теперь выглядит тот же самый класс `User`:

In [ ]:
from dataclasses import dataclass


@dataclass
class User:
    user_id: int
    name: str
    age: int
    email: str

И код, который был у нас дальше, после объявления класса, продолжает работать точно также, без каких-либо изменений. Соответственно, и IDE нам будет атрибуты подставлять, и статический анализатор кода ошибки типов указывать. Главное не забывать перед определением датакласса писать декоратор `@dataclass` и обязательно прописывать типы атрибутов.

Давайте приведу пример из разработки телеграм-ботов, чтобы немного заземлить теорию. Вот так, например, можно оформить класс Config, в экземпляре которого будут храниться конфигурационные данные для наших с вами ботов и данные для подключения к базе данных, которые мы будем получать сначала из переменных окружения:

In [3]:
from dataclasses import dataclass


@dataclass
class DatabaseConfig:
    db_host: str       # URL-адрес базы данных
    db_user: str       # Username пользователя базы данных
    db_password: str   # Пароль к базе данных
    database: str      # Название базы данных


@dataclass
class TgBot:
    token: str             # Токен для доступа к телеграм-боту
    admin_ids: list[int]   # Список id администраторов бота


@dataclass
class Config:
    tg_bot: TgBot
    db: DatabaseConfig

Если создать экземпляр класса Config:

In [ ]:
config = Config(tg_bot=TgBot(token=BOT_TOKEN,
                             admin_ids=ADMIN_IDS),
                db=DatabaseConfig(db_host=DB_HOST,
                                  db_user=DB_USER,
                                  db_password=DB_PASSWORD,
                                  database=DATABASE))

То, например, получить токен бота можно, будет так:

In [ ]:
token = config.tg_bot.token

А пароль к базе данных так:

In [ ]:
db_passw = config.db.db_password

Дополнительные материалы по датаклассам:

Статья на Хабре ["Введение в Data classes"][2]
Статья на Хабре ["9 причин использовать dataclasses в Python"][3]
[Материалы][5] Сергея Балакирева по DataClasses
Если на телеграмботах не планируете останавливаться, то один способ работы с типами переменных - это [pydantic][4], более крутая штука для работы с БД и валидацией данных.

[2]: https://habr.com/ru/post/415829/
[3]: https://habr.com/ru/company/otus/blog/650257/
[4]: https://www.youtube.com/watch?v=dOO3GmX6ukU
[5]: https://proproprogs.ru/python_oop/python-oop-vvedenie-v-python-data-classes-chast-1


# `__annotations__`

Где объекты хранят аннотации типов. Для этого существует специальный атрибут `__annotations__`, который хранит аннотации в виде словаря с ключами - названиями переменных и значениями - их типами.

**Пример**. Функция `get_string`, получающая на вход строку и число, а возвращающая строку.

In [6]:
def get_string(string: str, number: int) -> str:
    return string * number

Если обратиться к атрибуту `__annotations__` функции `get_string`: 

In [7]:
print(get_string.__annotations__)

{'string': <class 'str'>, 'number': <class 'int'>, 'return': <class 'str'>}


увидим следующий словарь:

```
{'string': <class 'str'>, 'number': <class 'int'>, 'return': <class 'str'>}
```

# Выводы

**Зачем вообще нужна аннотация типов?**

1. Выявление ошибок еще на этапе написания кода. С помощью `IDE` и статических анализаторов кода. А чем раньше выявлена ошибка, тем, в общем случае, дешевле ее исправить. 
2. Улучшение читаемости кода за счет явного указания, что ожидает и что возвращает функция или класс.
3. Упрощение разработки в IDE за счет подсказок и автодополнения. В частности, в `aiogram` большое количество классов, которые являются отображением методов Telegram Bot API, и довольно удобно не лезть в документацию, а прямо внутри хэндлера, при его написаниии, получать мини-справку по доступному функционалу.

**Важная особенность 1:**

Интерпретатор никак не проверяет аннотации типов. То есть код, при котором в функцию будет передан не тот тип данных, что указан в аннотации, может отработать и без ошибок. 

**Важная особенность 2:**

Переменные также можно аннотировать, причем необязательно присваивать им значения. Но надо иметь в виду, что присваивать значения переменным в программе нужно до их первого использования, иначе интерпретатор выкинет исключение `NameError`.

**Для аннотаций типов используются:**

1. Встроенные типы данных языка (`int`, `float`, `str`, `list`, `dict`, `tuple`...)
2. Объекты из модуля typing (`List`, `Dict`, `NamedTuple`, `Union`, `Optional`, `Any`, `NoReturn`...)
3. Пользовательские классы
4. Классы из сторонних модулей
5. Датаклассы из модуля dataclasses

**Примечание.** Аннотации типов хранятся в атрибуте __annotations__ объектов.